<a href="https://colab.research.google.com/github/s1250103/Public_mori-lab/blob/preprcing-learning/experiments/preprcing-learning/preprcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 開発用モジュール

In [1]:
import matplotlib.pyplot as plt
import cv2
def display_ndarray(image):
  img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  plt.imshow(img)
  plt.show()

In [2]:
from tensorflow.keras import layers, models, initializers, callbacks
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import plot_model

# [A-2]環境構築

In [3]:
#@title gdrive マウント
from google.colab import drive
from os import path
if not path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("Already confirm")


Mounted at /content/drive


In [4]:
#@title 一時的にファイルを置くディレクトリ'desk'を作成
import os
desk = '/content/desk'
if not os.path.exists(desk):
  os.mkdir(desk)
os.chdir(desk)

In [7]:
#@title 必要なファイルを'desk'にコピー

wants_paths = [
'/content/drive/MyDrive/colab/dence1223/key20201218.h5',
'/content/drive/MyDrive/colab/dence1223/normalTrainLabels.npz',
'/content/drive/MyDrive/colab/dence1223/normalTrainVideos.npz'
]

import shutil
for want in wants_paths:
  if not os.path.exists(os.path.join(desk, os.path.basename(want))):
    shutil.copy2(want, desk)
    print("get : ", want)
            

get :  /content/drive/MyDrive/colab/dence1223/normalTrainLabels.npz
get :  /content/drive/MyDrive/colab/dence1223/normalTrainVideos.npz


In [8]:
#@title データフォーマットを規定
class video_format:
  name = "video_format"
  # サンプリングされたCMデータの仕様
  playtime = "15秒"
  displaysize = "(any, any, RGB)"
  videoformat = "any"
  # モデルが扱うCMデータ(上のようなデータは、下のように変換される)
  HEIGHT = 45
  WIDTH = 80
  FRAME_SIZE = 30
  COLOR = "RGB"
  FPS = "2 (FRAME_SIZE / playtime)" # 定義ではなく上から導かれた値
  
format1 = video_format()

# [A-1]データ取得・整形

## [A-1-1]データ取得

In [ ]:
import os
import cv2

# ディレクトリ名とラベルとの対応
ruleLabel = {
    '0.other' : 0, 
    '1.food' : 1,
    '2.car' : 2,
    '3.cosme' : 3}
    
class DataWithLabel:
  data = "video data"
  label = "train label"
  def __init__(self, data, label):
    self.data = data
    self.label = label

def getData(path):
  dwls = []

  dir_classes = os.listdir(path)
  for dir_class in dir_classes:
    print(dir_class)
    path_dir_class = os.path.join(path, dir_class)
    videosInTheClass = os.listdir(path_dir_class)
    for videoInTheClass in videosInTheClass:
      path_videoInTheClass = os.path.join(path_dir_class, videoInTheClass)
      print(path_videoInTheClass) # 扱うビデオデータのパスを確認
      video_cvformat = cv2.VideoCapture(path_videoInTheClass)
      label = ruleLabel[dir_class]
      print("Attached the label :", label)

      dwl = DataWithLabel(video_cvformat, label)
      dwls.append(dwl)
      print("________")

  return dwls


In [12]:
import os
path_trainData = "/content/drive/MyDrive/train_data"
path_testData = "/content/drive/MyDrive/test_data"

if not os.path.isdir(path_trainData):
  print("no train data")
if not os.path.isdir(path_testData):
  print("no test data")

# 訓練データ取得
dwls_train = getData(path_trainData)
# 訓練データ取得
dwls_test = getData(path_testData)

## [A-1-2]データ整形

In [ ]:
import numpy as np
import math

def makeUp_data(dwls):
  for count_dwl, dwl in enumerate(dwls):
    print("loading : ", count_dwl)
    framesize = int(dwl.data.get(cv2.CAP_PROP_FRAME_COUNT)) # フレームの数
    cutTiming = math.floor(framesize / format1.FRAME_SIZE)

    cutTail = cutTiming * format1.FRAME_SIZE

    normalVideo = []
    count = 0
    # フレーム３０枚分を１つの動画から間伐する
    while count <= cutTail - 1:
      IO, frame = dwl.data.read()
      if (count % cutTiming) == 0: 
        if not IO:
          print("error in : ", count_dwl) 
          break
        else: 
          fixedFrame = cv2.resize(frame, dsize=(format1.WIDTH, format1.HEIGHT))
          fixedFrame = fixedFrame.flatten().astype(np.float32)/255.0
          normalVideo.append(fixedFrame)
      else:
        pass

      count += 1


    dwl.data = np.array(normalVideo)


In [ ]:
makeUp_data(dwls_train)
makeUp_data(dwls_test)

## [A-1-3]データ保存

In [18]:
def save_data(dwls, saveName):
  videos = []
  labels = []
  for dwl in dwls:
    videos.append(dwl.data.flatten())
    labels.append(dwl.label)
  
  name_videos = saveName + "Videos"
  name_labels = saveName + "Labels"

  np.savez(os.path.join(desk, name_videos), *videos)
  np.savez(os.path.join(desk, name_labels), *labels)

In [ ]:
save_data(dwls_train, "normalTrain")
save_data(dwls_test, "normalTest")

試しに読み込んでみる

In [13]:
import numpy as np

v = np.load('/content/desk/normalTrainVideos.npz')
oneVideo = v['arr_0']
# display_ndarray(oneVideo[10])

In [14]:
oneVideo.shape

(324000,)

In [ ]:
import numpy as np

v = np.load('/content/desk/normalTestVideos.npz')
oneVideo = v['arr_7']
display_ndarray(oneVideo[2])

# [A-3]モデルの作成

In [ ]:
#@title 入力データの情報を規定
colorSet = {'BW' : 1, 'RGB' : 3}

#入力層の設定
input_frameWIDTH = format1.WIDTH
input_frameHEIGHT = format1.HEIGHT
input_frameSIZE = format1.FRAME_SIZE
input_frameCOLOR = colorSet['RGB']
input_allPattern = input_frameWIDTH * input_frameHEIGHT * input_frameSIZE * input_frameCOLOR

print(
    input_frameWIDTH,
    input_frameHEIGHT,
    input_frameSIZE,
    input_frameCOLOR,
    input_allPattern )

80 45 30 3 324000


## [A-3-2]モデルの作成

In [ ]:
from tensorflow.keras import layers, models, initializers, callbacks
import tensorflow as tf
import numpy as np

seedKey = 20201218

np.random.seed(seedKey)
tf.random.set_seed(seedKey)

model = models.Sequential([
    # 入力層        
    layers.Reshape(
        (input_frameSIZE, input_frameHEIGHT, input_frameWIDTH,  input_frameCOLOR), 
        input_shape=(input_allPattern,),
        name='ENTRANCE' ),
    # 中間層
    layers.Flatten(
        name='RIVER' ),
    layers.Dense(
      1024, # 1024項
      activation='relu',
      name='DAM' ),
    # 出力層
    layers.Dense(4, activation='softmax', name='WATERSUPPLY')
])

name_model = 'key' + str(seedKey)
name_model = name_model + '.h5'
path_model = os.path.join('/content/desk/', name_model)
model.save(path_model)